Вот программа для финансового прогнозирования, которая извлекает данные из онлайн-источника и использует нейронные сети, чтобы делать прогнозы на следующие пять лет. Программа ориентирована на прогнозирование фондового рынка и отображает результаты в виде графики на экране. Я подробно опишу каждую команду.

In [ ]:
# Импорт зависимостей: импортируйте необходимые библиотеки, включая NumPy, Pandas, TensorFlow, matplotlib и yfinance.
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf

**Получение данных фондового рынка:** используйте **yf.download()** из библиотеки **yfinance** для получения исторических данных фондового рынка. Укажите символ биржевого тикера (**ticker**), дату начала (**start_date**) и дату окончания (**end_date**)

In [ ]:
# Получить данные фондового рынка от Yahoo Finance
ticker = 'AAPL'  # Биржевой символ
start_date = '2010-01-01'  # Дата начала для исторических данных
end_date = '2023-12-31'  # Дата окончания для исторических данных

Подготовьте набор данных: создайте **DataFrame** (df) для хранения исторических цен на акции.

In [ ]:
data = yf.download(ticker, start=start_date, end=end_date)

In [ ]:
# Подготовьте набор данных
df = pd.DataFrame(data['Close'])
df.reset_index(inplace=True)
df.columns = ['Date', 'Close']

In [ ]:
# При необходимости создайте дополнительные функции
# Разделите набор данных на наборы для обучения и тестирования.
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

In [ ]:
# Масштабировать данные
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data['Close'].values.reshape(-1, 1))
test_scaled = scaler.transform(test_data['Close'].values.reshape(-1, 1))


In [ ]:
# Определение обучающих последовательностей и меток
sequence_length = 30  # Length of input sequences
train_sequences = []
train_labels = []

for i in range(sequence_length, len(train_scaled)):
    train_sequences.append(train_scaled[i-sequence_length:i, 0])
    train_labels.append(train_scaled[i, 0])

train_sequences = np.array(train_sequences)
train_labels = np.array(train_labels)

In [ ]:
# Определите архитектуру нейронной сети
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(64, input_shape=(sequence_length, 1)))
model.add(tf.keras.layers.Dense(1))

In [ ]:
# Скомпилируйте модель
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Обучите модель
model.fit(train_sequences, train_labels, epochs=100, batch_size=32, verbose=1)


In [ ]:
# Делайте прогнозы на ближайшие пять лет
num_predictions = 5 * 252  # 252 торговых дня в году
prediction_input = test_scaled[-sequence_length:].reshape(1, -1, 1)
predictions = []

for _ in range(num_predictions):
    prediction = model.predict(prediction_input)
    predictions.append(prediction)
    prediction_input = np.append(prediction_input[:, 1:, :], prediction.reshape(1, 1, 1), axis=1)

predictions = np.array(predictions).reshape(-1, 1)
predictions = scaler.inverse_transform(predictions)

In [ ]:
# Визуализируйте прогнозы
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Close'], label='Действительный')
plt.plot(test_data['Date'], predictions[:len(test_data)], label='Предсказанный')
plt.xlabel('Дата')
plt.ylabel('Цена акций')
plt.title('Прогноз цен на акции')
plt.legend()
plt.show()

In [ ]:
# Сохраните модель
model.save('stock_forecasting_model.h5')